**1.코드1에서 저장한 주소 위도 경도 정보를 불러온다**

In [25]:
import warnings
warnings.filterwarnings('ignore')

from bs4 import BeautifulSoup
import pandas as pd
from urllib.request import urlopen

df2=pd.read_csv('address.csv',sep=',',encoding='cp949')

df2=df2.drop(df2.columns[0],axis='columns')#불필요한 열 제거하기

df2.head()

,주소,위도,경도
0,경기 양주시 백석읍 꿈나무로 199꿈나무도서관,37.792424,126.984524
1,대구 달성군 현풍면 부리 247 (상수도사업본부 달성사업소)(현풍중앙로 144-34...,35.699953,128.441276
2,경기 이천시 설성면 신필리산 88-5(전파연구소 입구),37.138773,127.544973
3,경기 포천시 삼육사로 2186번길 11-15선단보건지소,37.853336,127.157033
4,충북 음성군 금왕읍 무극로370 (금왕공설운동장내),36.962776,127.575536


**2.주소마다 행정구역 위치를 전처리하고, 지역 column을 추가**

In [26]:
a=[eachAddress.split()[0] for eachAddress in df2['주소']]


#행정구역 전처리 (같은 서울이여도 서울, 서울시, 서울특별시로 기록되는 경우가 있었음)
for i in range(len(a)):
    if '광역시' in a[i]:
        a[i]=a[i].replace('광역시','')
    elif '특별시' in a[i]:
        a[i]=a[i].replace('특별시','')
    elif '강서구' in a[i]:
        a[i]=a[i].replace('강서구','')
    elif '안산시' in a[i]:
        a[i]=a[i].replace('안산시','')
    elif '경기도' in a[i]:
        a[i]=a[i].replace('경기도','경기')
    elif '서울시' in a[i]:
        a[i]=a[i].replace('서울시','서울')
        
df2['지역']=a  # 17개의 행정구역별로 나눈다.
df2.head()
        

,주소,위도,경도,지역
0,경기 양주시 백석읍 꿈나무로 199꿈나무도서관,37.792424,126.984524,경기
1,대구 달성군 현풍면 부리 247 (상수도사업본부 달성사업소)(현풍중앙로 144-34...,35.699953,128.441276,대구
2,경기 이천시 설성면 신필리산 88-5(전파연구소 입구),37.138773,127.544973,경기
3,경기 포천시 삼육사로 2186번길 11-15선단보건지소,37.853336,127.157033,경기
4,충북 음성군 금왕읍 무극로370 (금왕공설운동장내),36.962776,127.575536,충북


**3. 주소마다 시군정보 역시 전처리하고, 시군 column을 추가한다.**

In [27]:
b=[eachAddress.split()[1] for eachAddress in df2['주소']]

for i in range(len(b)):
    if len(b[i])>=3:
        if b[i][2]=='시' or  b[i][2]=='군' or  b[i][2]=='구':
            b[i]=b[i][0]+b[i][1]
    if len(b[i])>=4:
        if b[i][3]=='시' or  b[i][3]=='군' or  b[i][3]=='구':
            b[i]=b[i][0]+b[i][1]+b[i][2]
    if '체육공원로' in b[i]:#부산강서고 체육공원로 dustfeel 사이트 자체 내에서 띄어쓰기를 안한 경우이다.
        b[i]='강서'
    if '보듬3로' in b[i] or '누리로' in b[i] or '조치원읍' in b[i]:#~로, 읍은 세종특별자치시 
        b[i]='세종'
    if '미추홀' in b[i]:
        b[i]='남구' #2018 이후로 인천 남구가 미추홀로 변경되어서 
        
df2['시군']=b
df2.head()


,주소,위도,경도,지역,시군
0,경기 양주시 백석읍 꿈나무로 199꿈나무도서관,37.792424,126.984524,경기,양주
1,대구 달성군 현풍면 부리 247 (상수도사업본부 달성사업소)(현풍중앙로 144-34...,35.699953,128.441276,대구,달성
2,경기 이천시 설성면 신필리산 88-5(전파연구소 입구),37.138773,127.544973,경기,이천
3,경기 포천시 삼육사로 2186번길 11-15선단보건지소,37.853336,127.157033,경기,포천
4,충북 음성군 금왕읍 무극로370 (금왕공설운동장내),36.962776,127.575536,충북,음성


**4.상세주소 column을 추가**

In [28]:
c=[eachAddress.split()[2] for eachAddress in df2['주소']]

for i in range(len(c)):
    if 4>len(c[i])>=3: 
        c[i]=c[i][0]+c[i][1]
    if len(c[i])>=4:
        c[i]=c[i][0]+c[i][1]+c[i][2]
        
df2['상세주소']=c
df2.head()

,주소,위도,경도,지역,시군,상세주소
0,경기 양주시 백석읍 꿈나무로 199꿈나무도서관,37.792424,126.984524,경기,양주,백석
1,대구 달성군 현풍면 부리 247 (상수도사업본부 달성사업소)(현풍중앙로 144-34...,35.699953,128.441276,대구,달성,현풍
2,경기 이천시 설성면 신필리산 88-5(전파연구소 입구),37.138773,127.544973,경기,이천,설성
3,경기 포천시 삼육사로 2186번길 11-15선단보건지소,37.853336,127.157033,경기,포천,삼육사
4,충북 음성군 금왕읍 무극로370 (금왕공설운동장내),36.962776,127.575536,충북,음성,금왕


**5. json에서 지정된 지역 이름과 다른 지역들의 이름을 바꿔준다.**

In [29]:
#리스트로 넣기 지도로 표시하기 위해 ID열 추가 

two_word_list=['서울','부산','인천','광주','대구','울산','대전']
more_detail=['성남','수원','안산','전주','창원','포항','청주','용인','고양','부천','안양','천안']

for n in df2.index:
    if df2.loc[n,'지역'] in two_word_list:
        df2.loc[n,'ID']=df2.loc[n,'지역']+' '+df2.loc[n,'시군']
    elif df2.loc[n,'시군'] in more_detail:
        df2.loc[n,'ID']=df2.loc[n,'시군']+' '+df2.loc[n,'상세주소']
    else:
        df2.loc[n,'ID']=df2.loc[n,'시군']


In [30]:
#부천지역의 경우에는 예외적으로 측정소가 부천시 ~로로 되어있어 수동으로 변경이 필요하다.
#부천 경인옛로-> 부천 소사,부천 심중로->부천 원미,부천 성오로->부천 오정으로 변경

for n in df2.index:
    if df2.loc[n,'ID']=='부천 경인옛':
        df2.loc[n,'ID']='부천 소사'
    elif df2.loc[n,'ID']=='부천 심중':
        df2.loc[n,'ID']='부천 원미'
    elif df2.loc[n,'ID']=='부천 성오':
        df2.loc[n,'ID']='부천 오정'

In [31]:
del df2['상세주소'] #필요없는 열 삭제 

In [32]:
df2.head()

,주소,위도,경도,지역,시군,ID
0,경기 양주시 백석읍 꿈나무로 199꿈나무도서관,37.792424,126.984524,경기,양주,양주
1,대구 달성군 현풍면 부리 247 (상수도사업본부 달성사업소)(현풍중앙로 144-34...,35.699953,128.441276,대구,달성,대구 달성
2,경기 이천시 설성면 신필리산 88-5(전파연구소 입구),37.138773,127.544973,경기,이천,이천
3,경기 포천시 삼육사로 2186번길 11-15선단보건지소,37.853336,127.157033,경기,포천,포천
4,충북 음성군 금왕읍 무극로370 (금왕공설운동장내),36.962776,127.575536,충북,음성,음성


In [33]:
df2.to_csv('plus ID.csv',sep=',',encoding='cp949')